In [ ]:
import requests
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [7]:
def daily_price(symbol, comparison_symbol, all_data=True, limit=1, aggregate=1, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df


def data():
    
    df = daily_price('BTC','USD')
    cleansing = (df['timestamp'] >= '2018-01-01') & (df['timestamp'] <= '2018-02-13')
    df = df.loc[cleansing]


    y_train=(df['close']-df['open'])*100/(df['open'])


    df = pd.read_csv('index.csv')
    cleansing = (df['Date'] >= '2018-01-01') & (df['Date'] <= '2018-02-13')
    df = df.loc[cleansing]
    x_train = (df['Close']-df['Open'])*100/(df['Open'])
   
    return x_train,y_train


def model(X, m,c):
    return tf.add(tf.multiply(X, m),c)

def regression(x_train,y_train):
    learning_rate = 0.0001
    training_epochs = 1000
    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.float32)


    m = tf.Variable(0.0, name="slope")
    c = tf.Variable(0.0, name="y-intercept")

    y_model = model(X, m,c)

    cost = tf.square(Y-y_model)
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    for epoch in range(training_epochs):
        for (x, y) in zip(x_train, y_train):
            sess.run(train_op, feed_dict={X: x, Y: y})

    m = sess.run(m)
    c = sess.run(c)
    print("Beta = ",m)
    sess.close()
    plt.scatter(x_train, y_train)
    y_learned = (x_train)*m+c
    plt.plot(x_train, y_learned, 'r')
    plt.show()
    grid(True)





regression(*data())


Beta =  -0.19304


ValueError: x and y must be the same size